# Cases cleanup

In [36]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
cases_csv = 'Resources/worldometer_coronavirus_daily_data.csv'
cases_df = pd.read_csv(cases_csv)
cases_df.head()

,date,country,cumulative_total_cases,daily_new_cases,active_cases,cumulative_total_deaths,daily_new_deaths
0,2020-2-15,Afghanistan,0.0,NaN,0.0,0.0,NaN
1,2020-2-16,Afghanistan,0.0,NaN,0.0,0.0,NaN
2,2020-2-17,Afghanistan,0.0,NaN,0.0,0.0,NaN
3,2020-2-18,Afghanistan,0.0,NaN,0.0,0.0,NaN
4,2020-2-19,Afghanistan,0.0,NaN,0.0,0.0,NaN


In [4]:
population_url = 'https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population'
tables = pd.read_html(population_url)
populations_df = tables[0]
populations_df.head()

,Rank,Country(or dependent territory),Population,% of world,Date,Source(official or United Nations)
0,1,China[b],1407751040,NaN,1 May 2021,National population clock[3]
1,2,India[c],1376399054,NaN,1 May 2021,National population clock[4]
2,3,United States[d],331449281,NaN,1 Apr 2020,2020 census result[5]
3,4,Indonesia,271350000,NaN,31 Dec 2020,National annual estimate[6]
4,5,Pakistan[e],225200000,NaN,1 Jul 2021,UN projection[2]


In [5]:
populations_df.rename(columns={'Country(or dependent territory)': 'country', 'Population':'population'}, inplace=True)
populations_df = populations_df[['country', 'population']]


In [6]:
populations_df.head()

,country,population
0,China[b],1407751040
1,India[c],1376399054
2,United States[d],331449281
3,Indonesia,271350000
4,Pakistan[e],225200000


In [7]:
for country in populations_df['country']:
    temp = country.split('[',1)[0]
    temp = temp.split('(',1)[0]
    country = temp

In [8]:
populations_df = populations_df.sort_values('country')
populations_df.head()

,country,population
184,Abkhazia[y],245424
42,Afghanistan,32890171
137,Albania,2845955
31,Algeria,44700000
207,American Samoa (US),56951


In [9]:
cases_df = cases_df.merge(populations_df, how='inner', on='country')

In [10]:
cases_df.rename(columns={'cumulative_total_cases': 'total_cases','cumulative_total_deaths': 'total_deaths'}, inplace=True)
cases_df = cases_df[['date', 'country', 'total_cases', 'active_cases', 'total_deaths', 'population']]

In [11]:
cases_df.head()

,date,country,total_cases,active_cases,total_deaths,population
0,2020-2-15,Afghanistan,0.0,0.0,0.0,32890171
1,2020-2-16,Afghanistan,0.0,0.0,0.0,32890171
2,2020-2-17,Afghanistan,0.0,0.0,0.0,32890171
3,2020-2-18,Afghanistan,0.0,0.0,0.0,32890171
4,2020-2-19,Afghanistan,0.0,0.0,0.0,32890171


In [12]:
unique = cases_df.drop_duplicates('country')

In [13]:
populations_df = pd.DataFrame({
    'country' : unique.country,
    'population' : unique.population
})

In [14]:
cases_df.drop(columns='population', inplace=True)

In [15]:
populations_df.head()

,country,population
0,Afghanistan,32890171
435,Albania,2845955
870,Algeria,44700000
1305,Andorra,78015
1740,Angola,32097671


In [16]:
populations_df.reset_index(drop=True, inplace=True)
populations_df.head()

,country,population
0,Afghanistan,32890171
1,Albania,2845955
2,Algeria,44700000
3,Andorra,78015
4,Angola,32097671


# Vaccines

In [17]:
vax_csv = 'Resources/country_vaccinations_by_manufacturer.csv'
vax_man_df = pd.read_csv(vax_csv)
vax_man_df.head()

,location,date,vaccine,total_vaccinations
0,Chile,2020-12-24,Pfizer/BioNTech,420
1,Chile,2020-12-25,Pfizer/BioNTech,5198
2,Chile,2020-12-26,Pfizer/BioNTech,8338
3,Chile,2020-12-27,Pfizer/BioNTech,8649
4,Chile,2020-12-28,Pfizer/BioNTech,8649


In [18]:
manufacturers = {
    'Pfizer/BioNTech' : ['PFZR'],
    'Sinovac' : ['SNVC'],
    'Moderna' : ['MDRN'],
    'Oxford/AstraZeneca' : ['OXFD'],
    'Johnson&Johnson' : ['JSJS']
    }

In [19]:
vax_man_df.replace(manufacturers, inplace=True)
vax_man_df.rename(columns={'vaccine': 'manufacturer_id'}, inplace=True)

In [20]:
vax_man_df.rename(columns={'location':'country'}, inplace=True)

In [21]:
manuf_df = pd.DataFrame(manufacturers).T

In [22]:
manuf_df.reset_index(inplace=True)

In [23]:
manuf_df.rename(columns={
    'index' : 'manufacturer_name',
    0 : 'manufacturer_id'
}, inplace=True)

In [24]:
manuf_df.head()

,manufacturer_name,manufacturer_id
0,Pfizer/BioNTech,PFZR
1,Sinovac,SNVC
2,Moderna,MDRN
3,Oxford/AstraZeneca,OXFD
4,Johnson&Johnson,JSJS


In [25]:
vax_man_df.head()


,country,date,manufacturer_id,total_vaccinations
0,Chile,2020-12-24,PFZR,420
1,Chile,2020-12-25,PFZR,5198
2,Chile,2020-12-26,PFZR,8338
3,Chile,2020-12-27,PFZR,8649
4,Chile,2020-12-28,PFZR,8649


In [26]:
cases_df.head()

,date,country,total_cases,active_cases,total_deaths
0,2020-2-15,Afghanistan,0.0,0.0,0.0
1,2020-2-16,Afghanistan,0.0,0.0,0.0
2,2020-2-17,Afghanistan,0.0,0.0,0.0
3,2020-2-18,Afghanistan,0.0,0.0,0.0
4,2020-2-19,Afghanistan,0.0,0.0,0.0


In [27]:
populations_df.head()

,country,population
0,Afghanistan,32890171
1,Albania,2845955
2,Algeria,44700000
3,Andorra,78015
4,Angola,32097671


# Adding Dataframes to Database

In [32]:
rds_connection_string = f"mdaxc6:databootcamp@coviddb.cgctrcdsmwc0.us-east-2.rds.amazonaws.com:5432/covid_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [33]:
populations_df.to_sql('countries', con=engine, if_exists='append', index=False)

In [37]:
pd.read_sql_query('select * from countries', con=engine).head()

,country,population
0,Afghanistan,32890171
1,Albania,2845955
2,Algeria,44700000
3,Andorra,78015
4,Angola,32097671


In [38]:
manuf_df.to_sql('manufacturer', con=engine, if_exists='append', index=False)

In [39]:
pd.read_sql_query('select * from manufacturer', con=engine).head()

,manufacturer_id,manufacturer_name
0,PFZR,Pfizer/BioNTech
1,SNVC,Sinovac
2,MDRN,Moderna
3,OXFD,Oxford/AstraZeneca
4,JSJS,Johnson&Johnson


In [40]:
vax_man_df.to_sql('vaccines', con=engine, if_exists='append', index=False)

In [41]:
pd.read_sql_query('select * from vaccines', con=engine).head()

,country,date,total_vaccinations,manufacturer_id
0,Chile,2020-12-24,420,PFZR
1,Chile,2020-12-25,5198,PFZR
2,Chile,2020-12-26,8338,PFZR
3,Chile,2020-12-27,8649,PFZR
4,Chile,2020-12-28,8649,PFZR


In [42]:
cases_df.to_sql('cases', con=engine, if_exists='append', index=False)

In [43]:
pd.read_sql_query('select * from cases', con=engine).head()

,country,date,total_cases,active_cases,total_deaths
0,Afghanistan,2020-02-15,0,0,0.0
1,Afghanistan,2020-02-16,0,0,0.0
2,Afghanistan,2020-02-17,0,0,0.0
3,Afghanistan,2020-02-18,0,0,0.0
4,Afghanistan,2020-02-19,0,0,0.0
